# Chapter1

#### import libraries

In [1]:
import numpy as np
import pandas as pd 
import re
import geonamescache
import unidecode
import json

#### import all headlines for the file

In [2]:
f = open('./data/headlines.txt',"r")
headlines = f.read().splitlines()
f.close()

#### create dictionaries for countries and cities

In [3]:
gc = geonamescache.GeonamesCache()

countries = [country["name"] for country in gc.get_countries().values()]
cities = [city["name"] for city in gc.get_cities().values()]
allcountries = gc.get_countries()

#### create mappings between accented and unicode countries and city names

In [5]:
country_accent_mapping = {
    unidecode.unidecode(country): country for country in countries
}

city_accent_mapping = {
    unidecode.unidecode(city): city for city in cities
}

##### create mappings between country code and country name 

In [6]:
country_to_code = {
    allcountries[country]['name']: country for country in allcountries
}

code_to_country = {
    country: allcountries[country]['name'] for country in allcountries
}

#### sort countries and cities by name lenght

In [7]:
unaccented_cities = list(city_accent_mapping.keys())
unaccented_countries = list(country_accent_mapping.keys())

unaccented_cities.sort(key=lambda item: (-len(item), item), reverse=False)
unaccented_countries.sort(key=lambda item: (-len(item), item), reverse=False)

#### create regexes for coutries and city names

In [8]:
_cities = "|".join(unaccented_cities)
regexCity = "\\b(" + _cities + ")\\b"
compiledReCity = re.compile(regexCity)

_countries = "|".join(unaccented_countries)
regexCountry = "\\b(" + _countries + ")\\b"
compiledReCountry = re.compile(regexCountry)

#### create dataframe with three columns

In [10]:
df = pd.DataFrame(columns =['headline', 'city', 'country']) 

#### create function extracting city and country from a headline

In [12]:
def city_and_country_from_a_headline(headline):
    
    '''
    find city name and country name in a given headline
    
    input: headline (string)
    output: mapping the headline to city(s) and/or countries (dict)
    
    '''
    # country found
    country_match = compiledReCountry.search(headline)
    if country_match:
        country = country_match.group(0)
    else:
        country = None
    
    # city found
    city_match = compiledReCity.search(headline)
    if city_match:
        city = city_match.group(0)
        # look up a country based on a accented city name
        if not country_match:
            _cities = gc.get_cities_by_name(city_accent_mapping[city])
            biggestCity = {}
            for c in _cities:
                for cityKey in c:
                    if len(biggestCity) == 0:
                        biggestCity = c[cityKey]
                    else:
                        if c[cityKey]['population'] > biggestCity['population']:
                            biggestCity = c[cityKey] 
            country = code_to_country[biggestCity['countrycode']]
    else:
        city = None
    
    return dict(headline=headline, country=country, city=city)    

#### test to see results for first 100 headlines

In [13]:
headline_cities_and_countries = [
    city_and_country_from_a_headline(headline) for headline in headlines
]
headline_cities_and_countries[-100:]

[{'headline': 'Zika case reported in Limoeiro',
  'country': 'Brazil',
  'city': 'Limoeiro'},
 {'headline': 'Ibadan tests new cure for Malaria',
  'country': 'Nigeria',
  'city': 'Ibadan'},
 {'headline': 'Gonorrhea has Arrived in Avon Lake',
  'country': 'United States',
  'city': 'Avon Lake'},
 {'headline': 'Pneumonia has not Left Kinshasa',
  'country': 'Democratic Republic of the Congo',
  'city': 'Kinshasa'},
 {'headline': 'Respiratory Syncytial Virus Hits Henderson',
  'country': 'United States',
  'city': 'Henderson'},
 {'headline': 'More Zika patients reported in Lakeland',
  'country': 'United States',
  'city': 'Lakeland'},
 {'headline': 'Malaria Vaccine is now Required in Winona',
  'country': 'United States',
  'city': 'Winona'},
 {'headline': 'More Patients in Canton are Getting Diagnosed with Norovirus',
  'country': 'United States',
  'city': 'Canton'},
 {'headline': 'Ronkonkoma is infested with Chickenpox',
  'country': 'United States',
  'city': 'Ronkonkoma'},
 {'headli

##### save to  json

In [22]:
save_file = "./data/headline_cities_and_countries.json"
with open(save_file, "w") as fout:
    fout.write(json.dumps(headline_cities_and_countries))

##### export dataframe to csv file

In [23]:
df.to_csv('./data/chapter1.csv', index=False)

#### save mappings

In [16]:
with open("./data/country_accent_mapping.json", "w") as fout:
    fout.write(json.dumps(country_accent_mapping))
with open("./data/city_accent_mapping.json", "w") as fout:
    fout.write(json.dumps(city_accent_mapping))
with open("./data/country_to_code.json", "w") as fout:
    fout.write(json.dumps(country_to_code))
with open("./data/code_to_country.json", "w") as fout:
    fout.write(json.dumps(code_to_country))

#### read df from json and print 10 first rows

In [21]:
import pandas as pd

data = pd.read_json("./data/headline_cities_and_countries.json")
data = data.replace({None: np.nan})

data.head(10)

,headline,country,city
0,Zika Outbreak Hits Miami,United States,Miami
1,Could Zika Reach New York City?,United States,New York City
2,First Case of Zika in Miami Beach,United States,Miami Beach
3,"Mystery Virus Spreads in Recife, Brazil",Brazil,Recife
4,Dallas man comes down with case of Zika,United States,Dallas
5,Trinidad confirms first Zika case,Bolivia,Trinidad
6,Zika Concerns are Spreading in Houston,United States,Houston
7,Geneve Scientists Battle to Find Cure,Switzerland,Geneve
8,The CDC in Atlanta is Growing Worried,United States,Atlanta
9,Zika Infested Monkeys in Sao Paulo,Brazil,Sao Paulo


# Chapter2